In [55]:
from hopfield_modern.hopfield_modern import Hopfield

In [56]:
import os
import matplotlib.pyplot as plt
import numpy as np
import random
from pprint import pprint
from tqdm.notebook import tqdm, tqdm_notebook
from typing import Callable
import itertools as it
import pandas as pd

In [57]:
def norm(prediction, data):
    t = prediction - data
    # print(np.linalg.norm(t))
    return 1 - np.linalg.norm(t)
    # return np.sum(np.abs(data))
    
    
def norm2(prediction, data):
    t = (prediction - data)
    return 1/(np.linalg.norm(t) + 1)


In [58]:
def softmax(data):
    t = np.exp(data - np.max(data))
    return t/np.sum(t)

In [59]:
def load_dir(dataset: list, path: str, label: bool, sort=True):
    dataset.sort(key=lambda e: e["id"])
    if dataset:
        index = dataset[-1]["id"]+1
    else:
        index = 0
    fnames = [(path+"/Red/"+i[:2]+"_Red.txt", path+"/Green/"+i[:2]+"_Green.txt",path+"/Blue/"+i[:2]+"_Blue.txt") for i in [i for i in os.walk(path)][1][2]]
    for fname in fnames:
        person = {"id": index}
        person["label"] = label
        with open(fname[0],"r") as rfile, open(fname[1],"r") as gfile, open(fname[2],"r") as bfile:
            person["r"] = [float(i) for i in rfile.readlines()[1:]]
            person["g"] = [float(i) for i in gfile.readlines()[1:]]
            person["b"] = [float(i) for i in bfile.readlines()[1:]]
        
        if sort:
            person["r"].sort()
            person["g"].sort()            
            person["b"].sort()
        
        index+=1
        dataset.append(person)
        
def load_dataset(dataset: list, path: str, pathpos: str, pathneg: str, sort=True):
    load_dir(dataset, path+"/"+pathpos, True, sort=sort)
    load_dir(dataset, path+"/"+pathneg, False, sort=sort)    
        
        
# def get_filter(dataset: list, filter: str):
#     if filter!="r" and filter!="g" and filter!="b":
#         raise ValueError('Wrong filter: must be "r", "g", "b"')
#     new_dataset = []
#     for person in dataset:
#         new_dataset.append({"id": person["id"], "label": person["label"], "data": person[filter]})
    
#     return new_dataset

def show_plot_by_filter(dataset: list, filter: str):
    plt.clf()
    if filter!="r" and filter!="g" and filter!="b":
        raise ValueError('Wrong filter: must be "r", "g", "b"')
    for i in np.arange(0,len(dataset), 1):
        color="green"
        if dataset[i]["label"]:
            color="red"
        for j in dataset[i][filter]:
            plt.plot(i,j, "o", markersize=0.5, color=color)
    plt.show()

    
def binarize_person(person: dict, precision=1e-3, up=1.8, down=0.2, radius=0, flatten=True, filters=3):
    person_data = np.zeros((3, int((up-down)//precision)+1), np.float32)[0:filters]
    
    dots  = (
        ((np.array(person["r"]).clip(down, up)-down)//precision).astype(np.uint32),
        ((np.array(person["g"]).clip(down, up)-down)//precision).astype(np.uint32),
        ((np.array(person["b"]).clip(down, up)-down)//precision).astype(np.uint32))[0:filters]

    
    for color_index in range(filters):
        for dot in dots[color_index]:
            person_data[color_index][max(0, dot-radius) : min(dot+radius+1, person_data.shape[1])].fill(1)
    
    if flatten:
        return {"id": person["id"], "label": person["label"], "data": person_data.flatten()}
    else:
        return {"id": person["id"], "label": person["label"], "data": person_data}

def binarize_person_by_filter(person: dict, precision=1e-3, up=1.8, down=0.2, radius=0, flatten=True, filter='g'):
    person_data = np.zeros((int((up-down)//precision)+1), np.float32)
    
    dots  = ((np.array(person[filter]).clip(down, up)-down)//precision).astype(np.uint32)

    
    for dot in dots:
        person_data[max(0, dot-radius) : min(dot+radius+1, person_data.shape[0])].fill(1)
    
    if flatten:
        return {"id": person["id"], "label": person["label"], "data": person_data.flatten()}
    else:
        return {"id": person["id"], "label": person["label"], "data": person_data}





def show_plot_avg_by_filter(dataset: list, filter: str):
    plt.clf()
    if filter!="r" and filter!="g" and filter!="b":
        raise ValueError('Wrong filter: must be "r", "g", "b"')
    for person in dataset:
        
        if person["label"]:
            color="red"
            x = 0
        else:
            color="green"
            x = 1
             
        for y in person[filter]:
            plt.plot(x,y, "o", markersize=0.5, color=color)
    plt.show()
    


In [60]:
def max_score_i(dataset: np.ndarray, X: np.ndarray, i: int, score_fn: Callable):
    if i==0:
        max_i = 1
    else:
        max_i = 0
    # print(f"len {len(dataset)}")
    for j in (k for k in range(len(dataset)) if k!=i):
        if score_fn(X,dataset[max_i]) <= score_fn(X,dataset[j]):
            max_i = j
            
    return max_i

In [61]:
dataset_full = []
load_dataset(dataset_full, "C:/Users/User/Desktop/Data/Data", "BC", "Control", sort=True)

In [62]:
N = 58 # const in test

pos = random.sample(dataset_full[:68], N//2)
neg = random.sample(dataset_full[68:], N//2)
dataset = np.concatenate([pos, neg]).tolist()
print(len(dataset))
assert sum([i['label'] for i in dataset])*2 == N

58


In [64]:
num_of_dims = 23 # const in test
filter = 'b'

result = []


image_scaling_test = np.linspace(10, 60, 10, dtype=int)
tests_per_generation = 1000



for image_scaling in tqdm_notebook(image_scaling_test):
    one_scale = []
    
    
    for test in range(tests_per_generation):
        one_test = []
        
        dataset_dim = np.array([{'data': random.sample(person[filter], num_of_dims),
                                'label': person['label'],
                                'id': person['id']
                                } for person in dataset])

        dataset_color = np.array([i['data'] for i in dataset_dim])
        dataset_color = np.array([np.concatenate([i]*image_scaling) for i in dataset_color])


        for p in dataset_color:
            p.sort()
            
        dataset_color -= dataset_color.min()
        dataset_color /= dataset_color.max()
        # dataset_color = softmax(dataset_color) # not!!!

        dataset_color = dataset_color*2-1

        for i in range(len(dataset_color)):
            
            _mask = np.ones(len(dataset_color), dtype=bool)
            _mask[i] = False

            _X = dataset_color[i]
            _images = dataset_color[_mask]
            
            model = Hopfield(_images.T.copy())
            out = model.run(_X, 16.0)
            max_i = max_score_i(dataset=dataset_color, X=out, i=i, score_fn=norm)
            
            
            if dataset_dim[max_i]["label"]==dataset_dim[i]["label"]:
                one_test.append(1)
            else:
                one_test.append(0)
    
        one_scale.append(np.average(one_test))
    result.append((image_scaling, dataset_color.shape[0]/dataset_color.shape[1], np.average(one_scale)))


pd.DataFrame(result).to_csv("results.csv", index=False)
        

  0%|          | 0/10 [00:00<?, ?it/s]